# Experiment 05: Data augmentation

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e05-8', 'e05-6', 'e05-4', 'e05-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = True
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e05'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
    
# Data augmentation
AUGMENT_FLIP = True # both up/down and left/right flips
AUGMENT_ROTATE = True # 90 degree rotations
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e05-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e05-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e05-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e05-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e05-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e05-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e05-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e05-8': {None: None},                                   # not enough bands in GE01
                         'e05-6': {None: None},                                    # not enough bands in GE01
                         'e05-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e05-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-pre_20210319-092729/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-gan_20210321-101445/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = True
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
    METRIC_MA_NIQE_HR = True

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
    METRIC_MA_NIQE_HR = True
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: True
NIQE: True
Perceptual Index (PI): True


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129221, 'val': 8113, 'test': 9293}
Total number of tiles: 146627


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 341.3 and sd 128.4 from json file @ data/toulon-laspezia-tiles/e05/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-' 
                                                  + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test],
                                   augment_flip=True,
                                   augment_rotate=True
                                  )
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2006480509994506
Output (float) min, max: -0.2000617970682984 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH, 
                                   metric_ma_niqe_hr=METRIC_MA_NIQE_HR)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
## OVERRIDE LAST!
#GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-149.h5'
FINAL_EPOCH_NAME = 'final_epoch-'
#FINAL_EPOCH_NAME = 'epoch_149-'
SENSORS = {'val': ['GE01']}

if METRIC_MA_NIQE_HR:
    HR_OR_SR = '-ma_niqe_HR-'
else:
    HR_OR_SR = ''
metric_ma_niqe_hr = METRIC_MA_NIQE_HR

if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    val_or_test = 'val'
    # SENSORS = {'val': ['GE01']}

    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    #if METRIC_MA:
    #    PRE_GAN = ['gan']
    #else:
    #    PRE_GAN = ['pre', 'gan']
    
    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
                
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_val[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + FINAL_EPOCH_NAME 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + HR_OR_SR + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

['gan'] ['GE01']
gan GE01
Computed 8 images in  136.7884030342102 seconds
Last image: {'G_pixel_loss': 0.00016043661162257195, 'G_perceptual_loss': 1.750753402709961, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.19866180419922, 'SSIM': 0.8012147545814514, 'Ma': 4.311977863311768, 'NIQE': 5.775315284729004, 'PI': 5.731668472290039}
Computed 8 images in  127.3764910697937 seconds
Last image: {'G_pixel_loss': 0.0001774327247403562, 'G_perceptual_loss': 0.683308482170105, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.1387825012207, 'SSIM': 0.9331005215644836, 'Ma': 4.248932838439941, 'NIQE': 4.339444637298584, 'PI': 5.045255661010742}
Computed 8 images in  122.40250205993652 seconds
Last image: {'G_pixel_loss': 0.0002751131250988692, 'G_perceptual_loss': 2.299079656600952, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.986042022705078, 'SSIM': 0.6858716607093811, 'Ma': 4.677267551422119, 'NIQE':

Computed 8 images in  120.98779964447021 seconds
Last image: {'G_pixel_loss': 0.0001269241765839979, 'G_perceptual_loss': 0.7089511156082153, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.05177307128906, 'SSIM': 0.9895067811012268, 'Ma': 2.9784905910491943, 'NIQE': 12.041583061218262, 'PI': 9.531545639038086}
Computed 8 images in  121.2337212562561 seconds
Last image: {'G_pixel_loss': 0.0002373509487370029, 'G_perceptual_loss': 0.8748219013214111, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.99199295043945, 'SSIM': 0.8430537581443787, 'Ma': 4.527480125427246, 'NIQE': 3.8607184886932373, 'PI': 4.666619300842285}
Computed 8 images in  121.43188667297363 seconds
Last image: {'G_pixel_loss': 0.00012830612831749022, 'G_perceptual_loss': 0.26967865228652954, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.28188705444336, 'SSIM': 0.9926209449768066, 'Ma': 2.8664159774780273, 'NIQE': 11.0192050933837

Computed 8 images in  120.85944104194641 seconds
Last image: {'G_pixel_loss': 0.00026256084674969316, 'G_perceptual_loss': 2.658385753631592, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.657493591308594, 'SSIM': 0.8228810429573059, 'Ma': 4.715516090393066, 'NIQE': 6.764448165893555, 'PI': 6.024466037750244}
Computed 8 images in  121.30117750167847 seconds
Last image: {'G_pixel_loss': 0.00040611872100271285, 'G_perceptual_loss': 3.7493152618408203, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.48900032043457, 'SSIM': 0.5924827456474304, 'Ma': 4.400812149047852, 'NIQE': 4.426204681396484, 'PI': 5.012696266174316}
Computed 8 images in  121.48818635940552 seconds
Last image: {'G_pixel_loss': 0.00035049294820055366, 'G_perceptual_loss': 2.579540967941284, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.02037239074707, 'SSIM': 0.6237937808036804, 'Ma': 4.57060432434082, 'NIQE': 4.294095039367676, '

Computed 8 images in  121.23428273200989 seconds
Last image: {'G_pixel_loss': 0.00021972051763441414, 'G_perceptual_loss': 0.9362905621528625, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.61843490600586, 'SSIM': 0.8636593818664551, 'Ma': 4.727044582366943, 'NIQE': 4.107723712921143, 'PI': 4.6903395652771}
Computed 8 images in  121.87008213996887 seconds
Last image: {'G_pixel_loss': 0.0001467141992179677, 'G_perceptual_loss': 1.0494273900985718, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 36.195865631103516, 'SSIM': 0.9367581009864807, 'Ma': 4.5913872718811035, 'NIQE': 6.228835105895996, 'PI': 5.818723678588867}
Computed 8 images in  121.42701172828674 seconds
Last image: {'G_pixel_loss': 0.00014187391207087785, 'G_perceptual_loss': 0.69078528881073, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 38.032169342041016, 'SSIM': 0.9795793294906616, 'Ma': 3.2320573329925537, 'NIQE': 9.0453519821167, 'P

Computed 8 images in  121.91352915763855 seconds
Last image: {'G_pixel_loss': 0.00014656438725069165, 'G_perceptual_loss': 1.6795761585235596, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.546485900878906, 'SSIM': 0.9260242581367493, 'Ma': 3.7319719791412354, 'NIQE': 5.958304405212402, 'PI': 6.113166332244873}
Computed 8 images in  121.80221104621887 seconds
Last image: {'G_pixel_loss': 0.00018496460688766092, 'G_perceptual_loss': 2.70070481300354, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.23783493041992, 'SSIM': 0.7603407502174377, 'Ma': 5.0539164543151855, 'NIQE': 3.7061591148376465, 'PI': 4.3261213302612305}
Computed 8 images in  121.25347328186035 seconds
Last image: {'G_pixel_loss': 0.00026971433544531465, 'G_perceptual_loss': 3.557915449142456, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.028711318969727, 'SSIM': 0.7521271705627441, 'Ma': 4.714019775390625, 'NIQE': 4.2404360771179

...

Computed 8 images in  120.03077960014343 seconds
Last image: {'G_pixel_loss': 0.00028467370430007577, 'G_perceptual_loss': 4.119622230529785, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.6746768951416, 'SSIM': 0.7456167936325073, 'Ma': 4.752557754516602, 'NIQE': 3.660846710205078, 'PI': 4.454144477844238}
Computed 8 images in  120.36720442771912 seconds
Last image: {'G_pixel_loss': 0.00028155159088782966, 'G_perceptual_loss': 3.916527032852173, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.749473571777344, 'SSIM': 0.7869324088096619, 'Ma': 4.804182529449463, 'NIQE': 4.128907203674316, 'PI': 4.662362098693848}
Computed 8 images in  120.21954321861267 seconds
Last image: {'G_pixel_loss': 0.0002260250475956127, 'G_perceptual_loss': 3.025782346725464, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.977275848388672, 'SSIM': 0.7652952671051025, 'Ma': 4.935803413391113, 'NIQE': 3.446251630783081, 'P

Computed 8 images in  120.07459020614624 seconds
Last image: {'G_pixel_loss': 0.0003484105982352048, 'G_perceptual_loss': 4.952151298522949, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.656341552734375, 'SSIM': 0.5242334604263306, 'Ma': 4.5341796875, 'NIQE': 4.089264869689941, 'PI': 4.777542591094971}
Computed 8 images in  119.90760731697083 seconds
Last image: {'G_pixel_loss': 0.0002594245597720146, 'G_perceptual_loss': 1.5761909484863281, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.898033142089844, 'SSIM': 0.8394153714179993, 'Ma': 5.063672065734863, 'NIQE': 4.898334980010986, 'PI': 4.917331695556641}
Computed 8 images in  120.27375912666321 seconds
Last image: {'G_pixel_loss': 0.00035494068288244307, 'G_perceptual_loss': 4.900876522064209, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.979795455932617, 'SSIM': 0.6862227916717529, 'Ma': 4.684082984924316, 'NIQE': 4.230668544769287, 'PI':

Computed 8 images in  121.05461072921753 seconds
Last image: {'G_pixel_loss': 0.00036539434222504497, 'G_perceptual_loss': 3.88436222076416, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.215373992919922, 'SSIM': 0.6148322820663452, 'Ma': 4.709545135498047, 'NIQE': 3.7147865295410156, 'PI': 4.502620697021484}
Computed 8 images in  120.93273448944092 seconds
Last image: {'G_pixel_loss': 0.0003113054553978145, 'G_perceptual_loss': 3.3788297176361084, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.58871078491211, 'SSIM': 0.6428082585334778, 'Ma': 4.554614543914795, 'NIQE': 4.001298427581787, 'PI': 4.723341941833496}
Computed 8 images in  120.33038926124573 seconds
Last image: {'G_pixel_loss': 0.0004396054137032479, 'G_perceptual_loss': 4.315700531005859, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.893033981323242, 'SSIM': 0.4978628158569336, 'Ma': 4.658210277557373, 'NIQE': 4.30607795715332, 'P

Computed 8 images in  120.09181761741638 seconds
Last image: {'G_pixel_loss': 0.0008104036678560078, 'G_perceptual_loss': 5.680203914642334, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 20.852859497070312, 'SSIM': 0.5597246289253235, 'Ma': 4.314245700836182, 'NIQE': 3.911710023880005, 'PI': 4.798732280731201}
Computed 8 images in  120.43299412727356 seconds
Last image: {'G_pixel_loss': 0.0010604487033560872, 'G_perceptual_loss': 5.782840728759766, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 19.42715835571289, 'SSIM': 0.5296116471290588, 'Ma': 4.342672348022461, 'NIQE': 4.127440929412842, 'PI': 4.8923845291137695}
Computed 8 images in  121.13046908378601 seconds
Last image: {'G_pixel_loss': 0.0002918585960287601, 'G_perceptual_loss': 0.37513962388038635, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.2593994140625, 'SSIM': 0.9399586915969849, 'Ma': 2.9277374744415283, 'NIQE': 10.972780227661133, 

Computed 8 images in  119.90793490409851 seconds
Last image: {'G_pixel_loss': 0.0009193585719913244, 'G_perceptual_loss': 5.6021318435668945, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 19.953895568847656, 'SSIM': 0.5506108999252319, 'Ma': 4.6003265380859375, 'NIQE': 4.500969886779785, 'PI': 4.950321674346924}
Computed 8 images in  120.51827430725098 seconds
Last image: {'G_pixel_loss': 0.0007752202800475061, 'G_perceptual_loss': 6.239398002624512, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 21.225135803222656, 'SSIM': 0.5384799838066101, 'Ma': 4.397874355316162, 'NIQE': 4.439978122711182, 'PI': 5.02105188369751}
Computed 8 images in  120.61250591278076 seconds
Last image: {'G_pixel_loss': 0.0009771374752745032, 'G_perceptual_loss': 7.096005916595459, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 19.50231170654297, 'SSIM': 0.5386949181556702, 'Ma': 4.47708797454834, 'NIQE': 4.407766342163086, 'PI

### 8.3 Evaluate every kth epoch

In [17]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'val'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_val[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                       per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots